In [1]:
# adding required packages
import findspark
import pprint
import matplotlib.pyplot as plt

In [2]:
#To make pyspark importable as a regular library
findspark.init()

In [3]:
#importing pyspark related package
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.functions import col, when, avg, round, rank
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, StringType

In [4]:
import pyspark.sql.functions as f

In [5]:
%run /notebook/dataproduct/assesment_nyc_job_posting/utils/spark_session.ipynb

Spark Session Imported


In [6]:
# Local System Configuration
# Total Memory = 16GB
# Total Cores = 10

def get_spark_conf():
    # Configure Spark settings
    spark_conf = SparkConf()
    spark_conf.set("spark.executor.instances", "4") # 4 instance per node

    # Set the number of executor cores
    spark_conf.set("spark.executor.cores", "1")  # Use 1 cores per executor

    # Set the executor memory
    spark_conf.set("spark.executor.memory", "1g")  # Use 1GB memory per executor

    # Set the driver memory
    spark_conf.set("spark.driver.memory", "2g")    # Use 2GB memory for the driver
    
    return spark_conf

In [7]:
#setting spark conf before creating spark session
spark_conf = get_spark_conf()

# Create a SparkSession with the configured settings
spark = SparkSession.builder.config(conf=spark_conf).appName("MySparkApp").getOrCreate()

# Listing all the spark conf
spark.sparkContext.getConf().getAll()

# setting spark conf for analysis
spark.conf.set('spark.sql.repl.eagerEval.enabled',True)

#reading dataset
df = spark.read.csv("/dataset/nyc-jobs.csv", header=True, inferSchema=True, escape='"')

# reducing the shuffle partition to 4 
# reason 1 data size is very less
# reason 2 to use all the availble cores
spark.conf.set('spark.sql.shuffle.partitions',4)

# remove punctuation

In [15]:
%run /notebook/dataproduct/assesment_nyc_job_posting/lib/data_preparation.ipynb

Data Preparation Functions Imported


In [16]:
#remove punctuation
#df = df.select('Preferred Skills', f.coalesce(f.regexp_replace("Preferred Skills", "[^\w\s]+", ""),f.lit('')).alias('preferred_skill_non_punch'))
Cleaned_df = text_preparation(df, 'Preferred Skills')

In [17]:
Cleaned_df.select('Preferred Skills','Preferred Skills_prepared','Preferred Skills_splitted').limit(5).show(vertical=True,truncate=False)

AnalysisException: "cannot resolve '`Preferred Skills_splitted`' given input columns: [Job Description, Process Date, Residency Requirement, Division/Work Unit, Agency, Preferred Skills_prepared, Recruitment Contact, # Of Positions, Title Code No, Salary Range To, Hours/Shift, Post Until, Full-Time/Part-Time indicator, Posting Type, Salary Range From, Posting Date, Business Title, Preferred Skills, Job Category, Additional Information, Minimum Qual Requirements, Level, Salary Frequency, Work Location, Work Location 1, Civil Service Title, To Apply, Posting Updated, Job ID];;\n'Project [Preferred Skills#27, Preferred Skills_prepared#554, 'Preferred Skills_splitted]\n+- Project [Job ID#10, Agency#11, Posting Type#12, # Of Positions#13, Business Title#14, Civil Service Title#15, Title Code No#16, Level#17, Job Category#18, Full-Time/Part-Time indicator#19, Salary Range From#20, Salary Range To#21, Salary Frequency#22, Work Location#23, Division/Work Unit#24, Job Description#25, Minimum Qu

In [22]:
spark.stop()